<a href="https://colab.research.google.com/github/Profe-Jose-Burgos/Multi-Bot/blob/main/chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Entrenamiento



In [1]:
import json

In [2]:
data = open("/content/tryouts.json").read()
intents = json.loads(data)

In [3]:
import nltk
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
words = []
classes = []
documents = []
ignore_words = ['?','!']

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
      x = nltk.word_tokenize(pattern)
      words.extend(x)
      documents.append((x, intent['tag']))

      if intent['tag'] not in classes:
        classes.append(intent['tag'])

words = [lemmatizer.lemmatize(x.lower()) for x in words if x not in ignore_words]

In [6]:
import pickle

In [7]:
pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))


training = []
output_empty = [0] * len(classes)

for doc in documents:   
    count = []    
    pattern_words = doc[0]    
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for w in words:
        count.append(1) if w in pattern_words else count.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([count, output_row])

In [8]:
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD

In [9]:
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-9-bfaf44b08e8b>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [10]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/300
6/6 [==============================] - 1s 3ms/step - loss: 2.1894 - accuracy: 0.1852
Epoch 2/300
6/6 [==============================] - 0s 2ms/step - loss: 2.0973 - accuracy: 0.1852
Epoch 3/300
6/6 [==============================] - 0s 2ms/step - loss: 1.9354 - accuracy: 0.3704
Epoch 4/300
6/6 [==============================] - 0s 3ms/step - loss: 1.9422 - accuracy: 0.2593
Epoch 5/300
6/6 [==============================] - 0s 3ms/step - loss: 1.8655 - accuracy: 0.2593
Epoch 6/300
6/6 [==============================] - 0s 3ms/step - loss: 1.7759 - accuracy: 0.3704
Epoch 7/300
6/6 [==============================] - 0s 3ms/step - loss: 1.8024 - accuracy: 0.2593
Epoch 8/300
6/6 [==============================] - 0s 3ms/step - loss: 1.7127 - accuracy: 0.3333
Epoch 9/300
6/6 [==============================] - 0s 2ms/step - loss: 1.7047 - accuracy: 0.3333
Epoch 10/300
6/6 [==============================] - 0s 2ms/step - loss: 1.6190 - accuracy: 0.4074
Epoch 11/300
6/6 [===========

#Chat bot

In [11]:
import random
import numpy as np
from tensorflow.keras.models import load_model

In [12]:
model = load_model('chatbot_model.h5')
intents = json.loads(open('/content/tryouts.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [13]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def calc_pred(sentence, model):
    p = bow(sentence, words,show_details=False)
    response = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(response) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chat(msg):
    ints = calc_pred(msg, model)
    response = getResponse(ints, intents)
    return response

In [20]:
fileToken = open('/content/tokenbot.json')

data = json.load(fileToken)

fileToken.close

<function TextIOWrapper.close()>

In [21]:
BOT_TOKEN = data["token"]

In [17]:
!pip install pyTelegramBotAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 KB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.9.0-py3-none-any.whl size=203107 sha256=d4f03d83779e9dac979f5dc90f737b2e35ad1c4e79edc7c6592c7c5d43f61718
  Stored in directory: /root/.cache/pip/wheels/95/a7/fd/f667b71e29e3f64f239f30195efc39de642e72f8a47a6fcd8d
Successfully built pyTelegramBotAPI


In [18]:
import telebot

In [22]:
bot = telebot.TeleBot(BOT_TOKEN)


@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    bot.reply_to(message, "Howdy, how are you doing?")


@bot.message_handler(func=lambda message: True)
def echo_all(message):
  txt = message.text
  print(txt)
  bot.reply_to(message,chat(txt)) 


 

bot.infinity_polling()

Hola
1/1 [==============================] - 0s 19ms/step


2023-01-24 21:24:06,883 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2023-01-24 21:24:06,886 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling
